In [2]:
import pandas as pd

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [3]:
base_url = 'https://bechdeltest.com/'

# open connection, grab page
client = urlopen(base_url)

# store html in variable
base_html = client.read()

# close connection
client.close()

In [4]:
# parse html
soup = BeautifulSoup(base_html, 'html.parser')

In [5]:
soup.h1

<h1 style="margin-bottom: 5px"><a href="/" style="text-decoration:none;">Bechdel Test Movie List</a></h1>

In [6]:
soup.p

<p>The <b>Bechdel Test</b>, sometimes called the <i>Mo Movie Measure</i> or <i>Bechdel Rule</i> is a simple test which names the following three criteria: (1) it has to have at least two women in it, who (2) who talk to each other, about (3) something besides a man. The test was popularized by <b>Alison Bechdel</b>'s comic <b>Dykes to Watch Out For</b>, in a 1985 strip called <a href="http://alisonbechdel.blogspot.com/2005/08/rule.html"><i>The Rule</i></a>. For a nice video introduction to the subject please check out <a href="http://www.feministfrequency.com/2009/12/the-bechdel-test-for-women-in-movies/">The Bechdel Test for Women in Movies</a> on <a href="http://www.feministfrequency.com/">feministfrequency.com</a>.</p>

In [7]:
# grab each movie
movies = soup.findAll('div', {'class': 'movie'})

In [8]:
len(movies)

200

In [9]:
movies[0]

<div class="movie"><a href="http://us.imdb.com/title/tt0437086/"><img alt="[[3]]" src="/static/pass.png" title="[There are two or more women in this movie and they talk to each other about something other than a man]"/></a>
<a href="/view/8642/alita:_battle_angel/" id="movie-8642">Alita: Battle Angel</a> <a href="/view/8642/alita:_battle_angel/" onclick="showComments('8642'); return false;"><img alt="[1 comment(s) available]" id="comment-img-8642" src="/static/comments.png" style="height: 10px; width: 10px;" title="1 comment"/></a> </div>

In [10]:
# movies object contains imdb id (within imdb link), Bechdel Test score, 'pass' or 'nopass', movie title in 3 
# different formats, and number of comments
movie = movies[0]

In [11]:
movie.a

<a href="http://us.imdb.com/title/tt0437086/"><img alt="[[3]]" src="/static/pass.png" title="[There are two or more women in this movie and they talk to each other about something other than a man]"/></a>

In [16]:
movie.a.img

<img alt="[[3]]" src="/static/pass.png" title="[There are two or more women in this movie and they talk to each other about something other than a man]"/>

In [18]:
movie.a.img['alt']

'[[3]]'

In [19]:
# yessss! the score!!! 
movie.a.img['alt'][2]

'3'

In [17]:
movies[3]

<div class="movie"><a href="http://us.imdb.com/title/tt5719748/"><img alt="[[1]]" src="/static/nopass.png" title="[There are two or more women in this movie, but they don't talk to each other]"/></a>
<a href="/view/8639/cold_pursuit/" id="movie-8639">Cold Pursuit</a> <a href="/view/8639/cold_pursuit/" onclick="showComments('8639'); return false;"><img alt="[1 comment(s) available]" id="comment-img-8639" src="/static/comments.png" style="height: 10px; width: 10px;" title="1 comment"/></a> </div>